In [1]:
# %%
"""
════════════════════════════════════════════════════════════════════════════════
PREDICCIÓN DE DEMANDA TURÍSTICA EN MENDOZA
════════════════════════════════════════════════════════════════════════════════

"""

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configuración
import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Estilo de gráficos
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print(" Configuración completa")
print(f" Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print(f" Semilla aleatoria: {RANDOM_STATE}")

 Configuración completa
 Fecha: 2025-11-12 00:42
 Semilla aleatoria: 42


In [2]:
# %%
"""
════════════════════════════════════════════════════════════════════════════════
PASO 1: EXPLORACIÓN Y ENTENDIMIENTO DE LOS DATOS
════════════════════════════════════════════════════════════════════════════════
Objetivo: Conocer la estructura, calidad y características del dataset antes
          de aplicar cualquier transformación o limpieza.
          

"""

# ═══════════════════════════════════════════════════════════════════════════
# CARGA DE DATOS
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "="*80)
print(" CARGA DE DATOS")
print("="*80)

# Cargar dataset
df = pd.read_csv('mendoza_turismo_final_filtrado.csv', sep=';', encoding='utf-8')

# Crear copia de seguridad para análisis posteriores
df_original = df.copy()

print(f" Dataset cargado exitosamente")
print(f"   • Dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas")
print(f"   • Memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# ═══════════════════════════════════════════════════════════════════════════
# INSPECCIÓN INICIAL
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-"*80)
print(" PRIMERAS FILAS DEL DATASET")
print("-"*80)
print(df.head(10))

print("\n" + "-"*80)
print(" INFORMACIÓN DE COLUMNAS")
print("-"*80)
df.info()

print("\n" + "-"*80)
print(" ESTADÍSTICAS DESCRIPTIVAS")
print("-"*80)
print(df.describe())

# ═══════════════════════════════════════════════════════════════════════════
# ANÁLISIS DE LA VARIABLE OBJETIVO
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "="*80)
print(" ANÁLISIS DE LA VARIABLE OBJETIVO: 'turistas'")
print("="*80)

print(f"\n Estadísticas básicas:")
print(df['turistas'].describe())

print(f"\n   • Media: {df['turistas'].mean():,.0f} turistas/mes")
print(f"   • Mediana: {df['turistas'].median():,.0f} turistas/mes")
print(f"   • Desviación estándar: {df['turistas'].std():,.0f}")
print(f"   • Mínimo: {df['turistas'].min():,.0f}")
print(f"   • Máximo: {df['turistas'].max():,.0f}")
print(f"   • Rango: {df['turistas'].max() - df['turistas'].min():,.0f}")

# Coeficiente de variación
cv = (df['turistas'].std() / df['turistas'].mean()) * 100
print(f"   • Coeficiente de variación: {cv:.1f}%")
if cv > 100:
    print(f"     → Alta variabilidad en los datos")
elif cv > 50:
    print(f"     → Variabilidad moderada")
else:
    print(f"     → Baja variabilidad")

# ═══════════════════════════════════════════════════════════════════════════
# IDENTIFICACIÓN DE TIPOS DE VARIABLES
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "="*80)
print(" CLASIFICACIÓN DE VARIABLES")
print("="*80)

# Variables numéricas (excluyendo objetivo)
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
if 'turistas' in numeric_features:
    numeric_features.remove('turistas')

# Variables categóricas
categorical_features = df.select_dtypes(include=['object']).columns.tolist()

# Eliminar columnas de fecha que no son predictoras directas
date_columns = ['indice_tiempo', 'fecha', 'mes_ano', 'indice_tiempo_dt']
for col in date_columns:
    if col in categorical_features:
        categorical_features.remove(col)
    if col in numeric_features:
        numeric_features.remove(col)

print(f"\n Variables numéricas: {len(numeric_features)}")
if len(numeric_features) <= 15:
    for i, feat in enumerate(numeric_features, 1):
        print(f"   {i:>2}. {feat}")
else:
    for i, feat in enumerate(numeric_features[:10], 1):
        print(f"   {i:>2}. {feat}")
    print(f"   ... y {len(numeric_features) - 10} más")

print(f"\n Variables categóricas: {len(categorical_features)}")
for i, feat in enumerate(categorical_features, 1):
    n_unique = df[feat].nunique()
    print(f"   {i}. {feat} ({n_unique} categorías)")

# ═══════════════════════════════════════════════════════════════════════════
# CALIDAD DE DATOS
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "="*80)
print(" ANÁLISIS DE CALIDAD DE DATOS")
print("="*80)

# Valores nulos
print(f"\n Valores nulos por columna:")
nulos = df.isnull().sum()
if nulos.sum() == 0:
    print("    No hay valores nulos en el dataset")
else:
    print(f"    Total de valores nulos: {nulos.sum():,}")
    nulos_df = pd.DataFrame({
        'Columna': nulos[nulos > 0].index,
        'Cantidad': nulos[nulos > 0].values,
        'Porcentaje': (nulos[nulos > 0].values / len(df) * 100).round(2)
    })
    print(nulos_df.to_string(index=False))
    
    # Identificar columnas completamente nulas
    null_cols = df.columns[df.isnull().all()].tolist()
    if null_cols:
        print(f"\n   🗑️ Columnas completamente nulas detectadas: {null_cols}")
        print(f"      → Estas columnas serán eliminadas en la preparación")
        df = df.drop(columns=null_cols)
        # Actualizar listas de features
        numeric_features = [f for f in numeric_features if f not in null_cols]
        categorical_features = [f for f in categorical_features if f not in null_cols]
        print(f"   ✅ Columnas eliminadas: {null_cols}")

# Valores duplicados
duplicados = df.duplicated().sum()
print(f"\n Filas duplicadas: {duplicados}")
if duplicados > 0:
    print(f"    {duplicados} filas duplicadas detectadas ({duplicados/len(df)*100:.2f}%)")
else:
    print(f"    No hay filas duplicadas")

# ═══════════════════════════════════════════════════════════════════════════
# ANÁLISIS DE OUTLIERS EN TARGET
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "="*80)
print(" ANÁLISIS DE OUTLIERS EN VARIABLE OBJETIVO")
print("="*80)

# Método IQR
Q1 = df['turistas'].quantile(0.25)
Q3 = df['turistas'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers_mask = (df['turistas'] < lower_bound) | (df['turistas'] > upper_bound)
n_outliers = outliers_mask.sum()

print(f"\n Método IQR (Rango Intercuartílico):")
print(f"   • Q1 (25%): {Q1:,.0f}")
print(f"   • Q3 (75%): {Q3:,.0f}")
print(f"   • IQR: {IQR:,.0f}")
print(f"   • Límite inferior: {lower_bound:,.0f}")
print(f"   • Límite superior: {upper_bound:,.0f}")
print(f"   • Outliers detectados: {n_outliers} ({n_outliers/len(df)*100:.1f}%)")

if n_outliers > 0:
    print(f"\n    Top 5 outliers superiores:")
    top_outliers = df[df['turistas'] > upper_bound].nlargest(5, 'turistas')
    
    # Seleccionar columnas disponibles para mostrar
    display_cols = ['turistas']
    if 'año' in df.columns:
        display_cols.insert(0, 'año')
    if 'mes' in df.columns:
        display_cols.insert(1, 'mes')
    if 'pais_origen' in df.columns:
        display_cols.append('pais_origen')
    if 'punto_entrada' in df.columns:
        display_cols.append('punto_entrada')
    
    available_cols = [col for col in display_cols if col in df.columns]
    print(top_outliers[available_cols].to_string(index=False))
    
    print(f"\n   💡 Decisión: MANTENER outliers")
    print(f"      → Pueden representar temporadas altas legítimas")
    print(f"      → El pipeline usará StandardScaler que maneja outliers")

# ═══════════════════════════════════════════════════════════════════════════
# ANÁLISIS CRÍTICO: REGISTROS CON CERO TURISTAS
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "="*80)
print(" IDENTIFICACIÓN DE REGISTROS PROBLEMÁTICOS")
print("="*80)

ceros_count = (df['turistas'] == 0).sum()
ceros_pct = ceros_count / len(df) * 100

print(f"\n📊 Registros con 0 turistas:")
print(f"   • Total: {ceros_count} ({ceros_pct:.2f}%)")

if ceros_count > 0:
    print(f"\n    Análisis detallado de los ceros:")
    
    # Distribución temporal
    if 'año' in df.columns:
        ceros_por_año = df[df['turistas'] == 0].groupby('año').size().sort_index()
        print(f"\n    Distribución temporal:")
        for año, count in ceros_por_año.items():
            total_año = (df['año'] == año).sum()
            pct_año = count / total_año * 100
            print(f"      • {año}: {count:>3} registros ({pct_año:>5.1f}% del año)")
        
        # Identificar período más afectado
        año_mas_ceros = ceros_por_año.idxmax()
        max_ceros = ceros_por_año.max()
        
        print(f"\n    Año más afectado: {año_mas_ceros} ({max_ceros} registros con cero)")
        
        if año_mas_ceros in [2020, 2021]:
            print(f"      → Causa probable: Pandemia COVID-19")
            print(f"      → Cierre de fronteras y restricciones sanitarias")
    
    # Distribución por categorías
    if categorical_features:
        cat_principal = categorical_features[0]
        ceros_por_cat = df[df['turistas'] == 0].groupby(cat_principal).size().sort_values(ascending=False)
        
        print(f"\n    Distribución por {cat_principal} (top 5):")
        for cat, count in ceros_por_cat.head(5).items():
            print(f"      • {cat}: {count} registros")
    
    # Ejemplos de registros con cero
    print(f"\n    Ejemplos de registros con 0 turistas:")
    ceros_sample = df[df['turistas'] == 0].head(5)
    
    sample_cols = ['turistas']
    if 'año' in df.columns:
        sample_cols.insert(0, 'año')
    if 'mes' in df.columns:
        sample_cols.insert(1, 'mes')
    if categorical_features:
        sample_cols.extend(categorical_features[:2])
    
    available_sample_cols = [col for col in sample_cols if col in df.columns]
    print(ceros_sample[available_sample_cols].to_string(index=False))
    
    print(f"\n    OBSERVACIÓN IMPORTANTE:")
    print(f"      Estos registros con 0 turistas:")
    print(f"      • Representan situaciones excepcionales (cierres, crisis)")
    print(f"      • NO reflejan la demanda turística normal")
    print(f"      • Pueden distorsionar el entrenamiento del modelo")
    print(f"      • Serán ELIMINADOS en la preparación de datos (Celda 3)")
    print(f"\n      Justificación:")
    print(f"      → El objetivo es predecir demanda en condiciones normales")
    print(f"      → No queremos que el modelo 'aprenda' cierres excepcionales")
    print(f"      → Esto mejorará la utilidad práctica del modelo")

else:
    print(f"    No hay registros con 0 turistas")


# ═══════════════════════════════════════════════════════════════════════════
# RESUMEN DE LA EXPLORACIÓN
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "="*80)
print(" RESUMEN DE LA EXPLORACIÓN")
print("="*80)

print(f"""
 DATASET CARGADO Y EXPLORADO:
   • Registros: {len(df):,}
   • Variables: {df.shape[1]} ({len(numeric_features)} numéricas + {len(categorical_features)} categóricas)
   • Variable objetivo: turistas (media={df['turistas'].mean():,.0f}, std={df['turistas'].std():,.0f})

 PROBLEMAS IDENTIFICADOS:
   • Outliers: {n_outliers} ({n_outliers/len(df)*100:.1f}%) → MANTENER
   • Registros con 0: {ceros_count} ({ceros_pct:.2f}%) → ELIMINAR en Celda 3
   • Valores nulos: {nulos.sum()} → Ya tratados
   • Duplicados: {duplicados} → No hay

 DISTRIBUCIÓN:
   • Rango: {df['turistas'].min():,.0f} - {df['turistas'].max():,.0f}
   • Coeficiente de variación: {cv:.1f}% ({'Alta' if cv > 100 else 'Moderada' if cv > 50 else 'Baja'} variabilidad)
   • Asimetría detectada (probablemente sesgada a la derecha)

""")

print("="*80)
print(" EXPLORACIÓN COMPLETADA")
print("="*80)


 CARGA DE DATOS
 Dataset cargado exitosamente
   • Dimensiones: 2,877 filas × 31 columnas
   • Memoria: 1.15 MB

--------------------------------------------------------------------------------
 PRIMERAS FILAS DEL DATASET
--------------------------------------------------------------------------------
  indice_tiempo               pais_origen            punto_entrada  turistas  \
0       2014-01                    Brasil       Aeropuerto Córdoba       654   
1       2014-01                     Chile       Aeropuerto Córdoba      1046   
2       2014-01    Ee.Uu, Canadá Y México       Aeropuerto Córdoba      1211   
3       2014-01  Europa Y Resto Del Mundo       Aeropuerto Córdoba       719   
4       2014-01          Resto De América       Aeropuerto Córdoba       763   
5       2014-01           Resto Del Mundo      Puerto Buenos Aires      2135   
6       2014-01                   Uruguay      Puerto Buenos Aires     14122   
7       2014-02                   Bolivia  Aeropuerto Bu

In [3]:
# %%
"""
════════════════════════════════════════════════════════════════════════════════
PASO 1B: LIMPIEZA Y NORMALIZACIÓN DE CATEGORÍAS
════════════════════════════════════════════════════════════════════════════════

"""

print("\n" + "="*80)
print("🧹 LIMPIEZA DE CATEGORÍAS")
print("="*80)

# ═══════════════════════════════════════════════════════════════════════════
# FUNCIÓN PARA LIMPIAR COLUMNA CATEGÓRICA
# ═══════════════════════════════════════════════════════════════════════════

def limpiar_columna_categorica(df, columna, mapeo_dict, nombre_display=""):
    """
    Limpia y normaliza categorías de una columna
    
    Args:
        df: DataFrame
        columna: Nombre de la columna a limpiar
        mapeo_dict: Diccionario {valor_original: valor_normalizado}
        nombre_display: Nombre para mostrar en logs
    
    Returns:
        DataFrame modificado (in-place)
    """
    if columna not in df.columns:
        print(f" Columna '{columna}' no existe en el dataset")
        return df
    
    nombre = nombre_display or columna
    
    print(f"\n{'─'*80}")
    print(f"🔧 LIMPIANDO: {nombre}")
    print(f"{'─'*80}")
    
    # Mostrar categorías originales
    valores_antes = df[columna].value_counts()
    categorias_antes = df[columna].nunique()
    
    print(f"\n Categorías originales en '{columna}':")
    print(f"   Total categorías únicas: {categorias_antes}")
    print(f"\n   Listado completo:")
    for i, cat in enumerate(sorted(df[columna].unique()), 1):
        count = (df[columna] == cat).sum()
        pct = count / len(df) * 100
        print(f"      {i:2d}. {cat:<45} ({count:>5} registros, {pct:>5.2f}%)")
    
    # Aplicar mapeo
    print(f"\n⏳ Aplicando {len(mapeo_dict)} reglas de normalización...")
    df[columna] = df[columna].replace(mapeo_dict)
    
    # Normalizar espacios
    df[columna] = df[columna].str.strip()
    
    # Verificar resultados
    valores_despues = df[columna].value_counts()
    categorias_despues = df[columna].nunique()
    categorias_eliminadas = categorias_antes - categorias_despues
    
    print(f"\n Normalización completada:")
    print(f"   • Categorías antes: {categorias_antes}")
    print(f"   • Categorías después: {categorias_despues}")
    print(f"   • Categorías eliminadas/unificadas: {categorias_eliminadas}")
    
    # Mostrar categorías después de limpieza
    print(f"\n Categorías normalizadas en '{columna}':")
    print(f"   Total categorías únicas: {categorias_despues}")
    print(f"\n   Listado:")
    for i, cat in enumerate(sorted(df[columna].unique()), 1):
        count = (df[columna] == cat).sum()
        pct = count / len(df) * 100
        print(f"      {i:2d}. {cat:<45} ({count:>5} registros, {pct:>5.2f}%)")
    
    # Mostrar cambios específicos
    if categorias_eliminadas > 0:
        print(f"\n Detalle de cambios aplicados:")
        cambios_aplicados = 0
        for orig, nuevo in mapeo_dict.items():
            if orig in valores_antes.index:
                count = valores_antes[orig]
                cambios_aplicados += 1
                print(f"   ✓ '{orig}' → '{nuevo}' ({count} registros)")
        
        if cambios_aplicados > 0:
            print(f"\n    Total de transformaciones: {cambios_aplicados}")
            registros_afectados = sum(valores_antes[k] for k in mapeo_dict.keys() if k in valores_antes.index)
            print(f"    Registros afectados: {registros_afectados} ({registros_afectados/len(df)*100:.2f}%)")
    else:
        print(f"\n    No se encontraron categorías para mapear (ya estaban limpias)")
    
    return df

# ═══════════════════════════════════════════════════════════════════════════
# MAPEO 1: PAÍS DE ORIGEN
# ═══════════════════════════════════════════════════════════════════════════

mapeo_pais_origen = {
    # ─────────────────────────────────────────────────────────────────────
    # GRUPO: EEUU, CANADÁ Y MÉXICO
    # ─────────────────────────────────────────────────────────────────────
    'Ee.Uu Y Canadá': 'EEUU, Canadá y México',
    'Ee.Uu, Canadá': 'EEUU, Canadá y México',
    'Ee.Uu Y Canada': 'EEUU, Canadá y México',
    'Ee.Uu, Canada': 'EEUU, Canadá y México',
    'EEUU Y Canadá': 'EEUU, Canadá y México',
    'EEUU, Canadá': 'EEUU, Canadá y México',
    'Ee.Uu, Canadá Y México': 'EEUU, Canadá y México',
    'Ee.Uu Y Canadá Y México': 'EEUU, Canadá y México',
    'EEUU, Canada Y Mexico': 'EEUU, Canadá y México',
    'EEUU Y Canada': 'EEUU, Canadá y México',
    'EE.UU Y Canadá': 'EEUU, Canadá y México',
    'EE.UU, Canadá': 'EEUU, Canadá y México',
    'EE.UU, Canadá Y México': 'EEUU, Canadá y México',
    
    # ─────────────────────────────────────────────────────────────────────
    # GRUPO: EUROPA Y RESTO DEL MUNDO
    # ─────────────────────────────────────────────────────────────────────
    'Europa': 'Europa y Resto del Mundo',
    'Europa Y Resto Del Mundo': 'Europa y Resto del Mundo',
    'Europa y Resto del Mundo': 'Europa y Resto del Mundo',
    'Europa Y Resto Mundo': 'Europa y Resto del Mundo',
    'EUROPA': 'Europa y Resto del Mundo',
    'EUROPA Y RESTO DEL MUNDO': 'Europa y Resto del Mundo',
    'Resto Del Mundo': 'Europa y Resto del Mundo',
    
    # ─────────────────────────────────────────────────────────────────────
    # GRUPO: RESTO DE AMÉRICA
    # ─────────────────────────────────────────────────────────────────────
    'Resto América': 'Resto de América',
    'Resto America': 'Resto de América',
    'Resto De América': 'Resto de América',
    'Resto De America': 'Resto de América',
    'RESTO AMÉRICA': 'Resto de América',
    'RESTO AMERICA': 'Resto de América',
    'Resto América ': 'Resto de América',
    ' Resto América': 'Resto de América',
    'Resto  América': 'Resto de América',
}

# Aplicar limpieza a pais_origen
if 'pais_origen' in df.columns:
    df = limpiar_columna_categorica(
        df, 
        'pais_origen', 
        mapeo_pais_origen,
        "PAÍS DE ORIGEN"
    )
else:
    print(" Columna 'pais_origen' no encontrada")

# ═══════════════════════════════════════════════════════════════════════════
# MAPEO 2: PUNTO DE ENTRADA
# ═══════════════════════════════════════════════════════════════════════════

mapeo_punto_entrada = {
    'Aeropuerto Bs As': 'Aeropuerto Buenos Aires',
    'Aeropuerto BA': 'Aeropuerto Buenos Aires',
    'Aerop. Buenos Aires': 'Aeropuerto Buenos Aires',
    'Paso Cristo Redentor ': 'Paso Cristo Redentor',
    ' Paso Cristo Redentor': 'Paso Cristo Redentor',
}

# Aplicar limpieza a punto_entrada
if 'punto_entrada' in df.columns:
    df = limpiar_columna_categorica(
        df, 
        'punto_entrada', 
        mapeo_punto_entrada,
        "PUNTO DE ENTRADA"
    )
else:
    print(" Columna 'punto_entrada' no encontrada")

# ═══════════════════════════════════════════════════════════════════════════
# RESUMEN FINAL
# ═══════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print(" LIMPIEZA DE CATEGORÍAS COMPLETADA")
print("="*80)

print(f"\n Resumen final del dataset:")
print(f"   • Total registros: {len(df):,}")

if 'pais_origen' in df.columns:
    print(f"\n    País de Origen:")
    print(f"      • Categorías únicas: {df['pais_origen'].nunique()}")
    print(f"      • Sin valores nulos: {df['pais_origen'].notna().sum()}")
    print(f"\n      Top 5:")
    for i, (cat, count) in enumerate(df['pais_origen'].value_counts().head(5).items(), 1):
        pct = count / len(df) * 100
        print(f"         {i}. {cat:<40} ({count:>5}, {pct:>5.2f}%)")

if 'punto_entrada' in df.columns:
    print(f"\n    Punto de Entrada:")
    print(f"      • Categorías únicas: {df['punto_entrada'].nunique()}")
    print(f"      • Sin valores nulos: {df['punto_entrada'].notna().sum()}")
    print(f"\n      Top 5:")
    for i, (cat, count) in enumerate(df['punto_entrada'].value_counts().head(5).items(), 1):
        pct = count / len(df) * 100
        print(f"         {i}. {cat:<40} ({count:>5}, {pct:>5.2f}%)")

print("\n" + "="*80)
print(" Dataset listo para análisis con categorías normalizadas")
print("="*80)


🧹 LIMPIEZA DE CATEGORÍAS

────────────────────────────────────────────────────────────────────────────────
🔧 LIMPIANDO: PAÍS DE ORIGEN
────────────────────────────────────────────────────────────────────────────────

 Categorías originales en 'pais_origen':
   Total categorías únicas: 13

   Listado completo:
       1. Bolivia                                       (  139 registros,  4.83%)
       2. Brasil                                        (  373 registros, 12.96%)
       3. Chile                                         (  463 registros, 16.09%)
       4. Ee.Uu Y Canadá                                (  139 registros,  4.83%)
       5. Ee.Uu, Canadá                                 (   96 registros,  3.34%)
       6. Ee.Uu, Canadá Y México                        (  138 registros,  4.80%)
       7. Europa                                        (  139 registros,  4.83%)
       8. Europa Y Resto Del Mundo                      (  234 registros,  8.13%)
       9. Paraguay              

In [4]:
# %%
"""
════════════════════════════════════════════════════════════════════════════════
PASO 2: PREPARACIÓN DE DATOS Y CONSTRUCCIÓN DEL PIPELINE
════════════════════════════════════════════════════════════════════════════════
Objetivo: Crear un pipeline completo de Scikit-learn que integre:
   1. Limpieza de datos (eliminación de registros con 0 turistas)
   2. Preprocesamiento de variables numéricas (imputación + escalado)
   3. Preprocesamiento de variables categóricas (imputación + encoding)
   4. Partición train/test temporal
   

"""

# ═══════════════════════════════════════════════════════════════════════════
# LIMPIEZA DE DATOS: ELIMINACIÓN DE REGISTROS CON 0 TURISTAS
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "="*80)
print("🧹 LIMPIEZA DE DATOS")
print("="*80)

print(f"\n📊 Dataset antes de limpieza:")
print(f"   • Total de registros: {len(df):,}")
print(f"   • Registros con 0 turistas: {(df['turistas'] == 0).sum():,}")
print(f"   • Porcentaje de ceros: {(df['turistas'] == 0).sum() / len(df) * 100:.2f}%")

# Análisis previo a eliminación
ceros_count_before = (df['turistas'] == 0).sum()

if ceros_count_before > 0:
    print(f"\n🔍 Análisis de registros a eliminar:")
    
    # Estadísticas de los ceros
    df_ceros = df[df['turistas'] == 0]
    
    if 'año' in df.columns:
        print(f"\n   📅 Distribución temporal de los ceros:")
        ceros_por_año = df_ceros.groupby('año').size()
        for año, count in ceros_por_año.items():
            total_año = (df['año'] == año).sum()
            pct = count / total_año * 100
            print(f"      • {año}: {count} registros ({pct:.1f}% del año)")
    
    if categorical_features:
        cat_principal = categorical_features[0]
        print(f"\n   📊 Distribución por {cat_principal} (top 5):")
        ceros_por_cat = df_ceros.groupby(cat_principal).size().sort_values(ascending=False).head(5)
        for cat, count in ceros_por_cat.items():
            print(f"      • {cat}: {count} registros")
    
    print(f"\n💡 DECISIÓN: Eliminar todos los registros con 0 turistas")
    print(f"\n   Justificación:")
    print(f"   ✓ Representan situaciones excepcionales (cierres, pandemias)")
    print(f"   ✓ NO reflejan la demanda turística normal operativa")
    print(f"   ✓ Distorsionarían el entrenamiento y las métricas")
    print(f"   ✓ El objetivo es predecir demanda en condiciones normales")
    print(f"   ✓ Mejora la utilidad práctica del modelo")
    
    # Eliminar registros con 0 turistas
    df_clean = df[df['turistas'] > 0].copy()
    
    print(f"\n✅ Limpieza ejecutada:")
    print(f"   • Registros eliminados: {len(df) - len(df_clean):,}")
    print(f"   • Registros restantes: {len(df_clean):,} ({len(df_clean)/len(df)*100:.1f}%)")
    
    # Actualizar dataset
    df = df_clean
    
    # Estadísticas después de limpieza
    print(f"\n📊 Dataset después de limpieza:")
    print(f"   • Nueva media: {df['turistas'].mean():,.0f} turistas")
    print(f"   • Nueva mediana: {df['turistas'].median():,.0f} turistas")
    print(f"   • Nueva desv. estándar: {df['turistas'].std():,.0f}")
    print(f"   • Nuevo rango: {df['turistas'].min():,.0f} - {df['turistas'].max():,.0f}")
    
    # Comparación antes/después
    print(f"\n📊 Impacto de la limpieza:")
    print(f"   • Cambio en media: {df['turistas'].mean() - df_original['turistas'].mean():+,.0f} turistas")
    print(f"   • Cambio en std: {df['turistas'].std() - df_original['turistas'].std():+,.0f}")
    
else:
    print(f"   ✅ No hay registros con 0 turistas para eliminar")

# ═══════════════════════════════════════════════════════════════════════════
# SEPARACIÓN FEATURES Y TARGET
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "="*80)
print("📊 PREPARACIÓN DE VARIABLES")
print("="*80)

X = df[numeric_features + categorical_features].copy()
y = df['turistas'].copy()

print(f"✅ Features (X): {X.shape}")
print(f"✅ Target (y): {y.shape}")
print(f"\n   Variables predictoras:")
print(f"      • Numéricas: {len(numeric_features)}")
print(f"      • Categóricas: {len(categorical_features)}")

# Verificación de integridad
print(f"\n🔍 Verificación de integridad:")
print(f"   • Valores nulos en X: {X.isnull().sum().sum()}")
print(f"   • Valores nulos en y: {y.isnull().sum()}")
print(f"   • Valores cero en y: {(y == 0).sum()}")
print(f"   • Mínimo en y: {y.min():,.0f}")
print(f"   • Máximo en y: {y.max():,.0f}")

if (y == 0).sum() == 0:
    print(f"   ✅ Sin ceros en variable objetivo - Dataset limpio")

# ═══════════════════════════════════════════════════════════════════════════
# ESTRATEGIA DE PARTICIÓN
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "="*80)
print("✂️ ESTRATEGIA DE PARTICIÓN")
print("="*80)

print("""
Estrategia elegida: SPLIT TEMPORAL (80/20)

Justificación:
   • Los datos son series temporales (mensuales)
   • Queremos simular predicción del futuro con datos del pasado
   • Train/Test respeta el orden cronológico
   • Evita data leakage temporal

¿Por qué temporal y no aleatorio?
   ✓ En datos temporales, el orden importa
   ✓ Split aleatorio mezclaría pasado y futuro (data leakage)
   ✓ Queremos evaluar cómo predice el modelo en períodos futuros
   ✓ Es más realista para poner el modelo en producción

Distribución:
   • Train: 80% primeros datos (aprende de historia)
   • Test: 20% últimos datos (evalúa en futuro)
""")

# Split temporal
split_index = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

print(f"✅ Split temporal ejecutado:")
print(f"   • Train set: {len(X_train):,} registros ({len(X_train)/len(X)*100:.1f}%)")
print(f"   • Test set: {len(X_test):,} registros ({len(X_test)/len(X)*100:.1f}%)")

# Estadísticas train/test
print(f"\n📊 Estadísticas de la variable objetivo por conjunto:")
print(f"\n   TRAIN:")
print(f"      • Media: {y_train.mean():,.0f} ± {y_train.std():,.0f}")
print(f"      • Mediana: {y_train.median():,.0f}")
print(f"      • Rango: {y_train.min():,.0f} - {y_train.max():,.0f}")
print(f"      • Ceros: {(y_train == 0).sum()}")

print(f"\n   TEST:")
print(f"      • Media: {y_test.mean():,.0f} ± {y_test.std():,.0f}")
print(f"      • Mediana: {y_test.median():,.0f}")
print(f"      • Rango: {y_test.min():,.0f} - {y_test.max():,.0f}")
print(f"      • Ceros: {(y_test == 0).sum()}")

# Análisis de diferencia train/test
diff_mean = ((y_test.mean() - y_train.mean()) / y_train.mean()) * 100
print(f"\n   📊 Comparación:")
print(f"      • Diferencia en media: {diff_mean:+.2f}%")

if abs(diff_mean) > 20:
    print(f"      ⚠️ Gran diferencia entre train y test")
    print(f"         → Esto es NORMAL en split temporal (tendencias temporales)")
    print(f"         → El modelo aprenderá de un período y predecirá otro")
elif abs(diff_mean) > 10:
    print(f"      📊 Diferencia moderada entre train y test")
    print(f"         → Esperado en datos temporales con tendencias")
else:
    print(f"      ✅ Distribuciones similares entre train y test")

# Verificar que no hay ceros
if (y_train == 0).sum() == 0 and (y_test == 0).sum() == 0:
    print(f"\n   ✅ CONFIRMACIÓN: Sin ceros en train ni test")
    print(f"      → La limpieza de datos fue exitosa")

# ═══════════════════════════════════════════════════════════════════════════
# CONSTRUCCIÓN DEL PIPELINE DE PREPROCESAMIENTO
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "="*80)
print("🔧 CONSTRUCCIÓN DEL PIPELINE DE SCIKIT-LEARN")
print("="*80)

print("""
Pipeline de preprocesamiento con ColumnTransformer:

¿Qué es un pipeline?
   • Secuencia de transformaciones que se aplican automáticamente
   • Garantiza que train y test se procesan EXACTAMENTE igual
   • Evita data leakage (los parámetros se aprenden solo del train)
   • Facilita reproducibilidad y deployment

Estructura del pipeline:

1️⃣ VARIABLES NUMÉRICAS:
   a) Imputación: SimpleImputer (strategy='median')
      → Rellena valores faltantes con la mediana del train
      → ¿Por qué mediana? Es robusta a outliers
      → Se calcula SOLO en train, se aplica en train y test
   
   b) Escalado: StandardScaler
      → Normaliza a media=0, desviación estándar=1
      → ¿Por qué? Modelos como Ridge son sensibles a escala
      → Fórmula: (x - media) / std
      → Se calcula SOLO en train, se aplica en train y test

2️⃣ VARIABLES CATEGÓRICAS:
   a) Imputación: SimpleImputer (strategy='constant', fill_value='missing')
      → Rellena valores nulos con la categoría 'missing'
      → Crea una categoría explícita para datos faltantes
   
   b) Encoding: OneHotEncoder (handle_unknown='ignore')
      → Transforma categorías en columnas binarias (0/1)
      → Ejemplo: país=['Chile', 'Brasil'] → [país_Chile, país_Brasil]
      → handle_unknown='ignore': si test tiene categoría nueva, la ignora

3️⃣ INTEGRACIÓN CON COLUMNTRANSFORMER:
   • Aplica transformaciones en paralelo por tipo de variable
   • Concatena resultados en un solo array
   • Compatible con cualquier modelo de Scikit-learn
   • Usa todos los cores disponibles (n_jobs=-1)
""")

# Transformador para variables numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

print("✅ Transformador numérico creado:")
print("   1. SimpleImputer(strategy='median')")
print("   2. StandardScaler()")

# Transformador para variables categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

print("\n✅ Transformador categórico creado:")
print("   1. SimpleImputer(strategy='constant', fill_value='missing')")
print("   2. OneHotEncoder(handle_unknown='ignore')")

# ColumnTransformer combina ambos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop',  # Descartar columnas no especificadas
    n_jobs=-1  # Paralelizar transformaciones
)

print("\n✅ ColumnTransformer creado:")
print(f"   • Transformador numérico: {len(numeric_features)} variables")
print(f"   • Transformador categórico: {len(categorical_features)} variables")
print(f"   • Paralelización: n_jobs=-1 (todos los cores)")

# ═══════════════════════════════════════════════════════════════════════════
# AJUSTE Y VERIFICACIÓN DEL PIPELINE
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-"*80)
print("🧪 AJUSTE Y VERIFICACIÓN DEL PIPELINE")
print("-"*80)

print(f"\n⏳ Ajustando preprocessor en datos de train...")
print(f"   (Aprendiendo parámetros: mediana, media, std, categorías...)")

# Ajustar preprocessor (aprende parámetros del train)
preprocessor.fit(X_train)

print(f"✅ Preprocessor ajustado exitosamente")

# Transformar ambos conjuntos
print(f"\n⏳ Transformando datos...")
X_train_transformed = preprocessor.transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

print(f"\n✅ Transformación completada:")
print(f"\n   Train:")
print(f"      • Shape original: {X_train.shape}")
print(f"      • Shape transformado: {X_train_transformed.shape}")
print(f"      • Tipo: {type(X_train_transformed)}")

print(f"\n   Test:")
print(f"      • Shape original: {X_test.shape}")
print(f"      • Shape transformado: {X_test_transformed.shape}")
print(f"      • Tipo: {type(X_test_transformed)}")

# Calcular dimensionalidad
n_features_out = X_train_transformed.shape[1]
n_features_cat_created = n_features_out - len(numeric_features)

print(f"\n📊 Análisis de dimensionalidad:")
print(f"   • Features numéricas: {len(numeric_features)}")
print(f"   • Features categóricas originales: {len(categorical_features)}")
print(f"   • Columnas creadas por one-hot: {n_features_cat_created}")
print(f"   • Total features transformadas: {n_features_out}")

if n_features_out > 100:
    print(f"   ⚠️ Dimensionalidad alta (>{100}) - puede ralentizar entrenamiento")
elif n_features_out > 50:
    print(f"   📊 Dimensionalidad moderada (50-100)")
else:
    print(f"   ✅ Dimensionalidad baja (<50) - óptima para modelado")

# Verificar que no hay NaN o infinitos
print(f"\n🔍 Verificación de calidad:")
print(f"   • NaN en train transformado: {np.isnan(X_train_transformed).sum()}")
print(f"   • NaN en test transformado: {np.isnan(X_test_transformed).sum()}")
print(f"   • Inf en train transformado: {np.isinf(X_train_transformed).sum()}")
print(f"   • Inf en test transformado: {np.isinf(X_test_transformed).sum()}")

if (np.isnan(X_train_transformed).sum() == 0 and 
    np.isnan(X_test_transformed).sum() == 0 and
    np.isinf(X_train_transformed).sum() == 0 and
    np.isinf(X_test_transformed).sum() == 0):
    print(f"   ✅ Datos transformados limpios (sin NaN ni infinitos)")

# ═══════════════════════════════════════════════════════════════════════════
# VISUALIZACIÓN DE LA ESTRUCTURA DEL PIPELINE
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-"*80)
print("📊 ESTRUCTURA DEL PIPELINE")
print("-"*80)

from sklearn import set_config
set_config(display='diagram')

print("\n🔧 Diagrama del preprocesador:")
print(preprocessor)

# Mostrar resumen textual
print("\n📋 Resumen del pipeline:")
print(f"""
ColumnTransformer(
   transformers=[
      ('num', Pipeline([
         SimpleImputer(strategy='median'),
         StandardScaler()
      ]), {len(numeric_features)} variables numéricas),
      
      ('cat', Pipeline([
         SimpleImputer(strategy='constant'),
         OneHotEncoder(handle_unknown='ignore')
      ]), {len(categorical_features)} variables categóricas)
   ],
   remainder='drop',
   n_jobs=-1
)
""")

# ═══════════════════════════════════════════════════════════════════════════
# RESUMEN DE LA PREPARACIÓN
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "="*80)
print("📋 RESUMEN DE LA PREPARACIÓN")
print("="*80)

print(f"""
✅ LIMPIEZA COMPLETADA:
   • Registros eliminados: {ceros_count_before} (0 turistas)
   • Registros finales: {len(df):,}
   • Sin ceros en variable objetivo

✅ SPLIT TEMPORAL EJECUTADO:
   • Train: {len(X_train):,} ({len(X_train)/len(X)*100:.1f}%)
   • Test: {len(X_test):,} ({len(X_test)/len(X)*100:.1f}%)
   • Sin ceros en train ni test

✅ PIPELINE CONSTRUIDO Y AJUSTADO:
   • Preprocessor fitted en train
   • Transformaciones aplicadas en train y test
   • Dimensionalidad: {X_train.shape[1]} → {n_features_out}
   • Sin NaN ni infinitos

✅ DATOS LISTOS PARA MODELADO:
   • X_train: {X_train.shape}
   • X_test: {X_test.shape}
   • y_train: {y_train.shape}
   • y_test: {y_test.shape}
   • preprocessor: Listo para integrar con modelos

🎯 PRÓXIMOS PASOS:
   1. Definir modelos (Ridge, Random Forest, Gradient Boosting)
   2. Crear pipelines completos (preprocessor + modelo)
   3. Entrenar con validación cruzada
   4. Evaluar y comparar modelos
   5. Seleccionar el mejor
""")

print("="*80)
print("✅ PREPARACIÓN COMPLETADA - PIPELINE LISTO PARA MODELADO")
print("="*80)


🧹 LIMPIEZA DE DATOS

📊 Dataset antes de limpieza:
   • Total de registros: 2,877
   • Registros con 0 turistas: 389
   • Porcentaje de ceros: 13.52%

🔍 Análisis de registros a eliminar:

   📅 Distribución temporal de los ceros:
      • 2020: 207 registros (75.0% del año)
      • 2021: 182 registros (65.9% del año)

   📊 Distribución por pais_origen (top 5):
      • Europa y Resto del Mundo: 102 registros
      • Chile: 69 registros
      • EEUU, Canadá y México: 51 registros
      • Brasil: 51 registros
      • Resto de América: 51 registros

💡 DECISIÓN: Eliminar todos los registros con 0 turistas

   Justificación:
   ✓ Representan situaciones excepcionales (cierres, pandemias)
   ✓ NO reflejan la demanda turística normal operativa
   ✓ Distorsionarían el entrenamiento y las métricas
   ✓ El objetivo es predecir demanda en condiciones normales
   ✓ Mejora la utilidad práctica del modelo

✅ Limpieza ejecutada:
   • Registros eliminados: 389
   • Registros restantes: 2,488 (86.5%)

📊 D

In [5]:
# %%
"""
════════════════════════════════════════════════════════════════════════════════
PASO 3: COMPARACIÓN DE MODELOS
════════════════════════════════════════════════════════════════════════════════
Objetivo: Evaluar 3 modelos diferentes y seleccionar el mejor

Modelos a comparar:
   1. Ridge Regression (lineal con regularización L2)
   2. Random Forest (ensemble de árboles)
   3. Gradient Boosting (boosting secuencial)

Estrategia de evaluación:
   • Validación cruzada (5 folds) en train
   • Evaluación final en test
   • Métricas: RMSE, MAE, R²
"""

# ═══ DEFINICIÓN DE MODELOS CON PIPELINES COMPLETOS ═══
print("\n" + "="*80)
print(" DEFINICIÓN DE MODELOS")
print("="*80)

# Crear pipelines completos (preprocesamiento + modelo)
models = {
    'Ridge Regression': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', Ridge(alpha=1.0, random_state=RANDOM_STATE))
    ]),
    
    'Random Forest': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(
            n_estimators=100,
            max_depth=20,
            min_samples_split=5,
            random_state=RANDOM_STATE,
            n_jobs=-1
        ))
    ]),
    
    'Gradient Boosting': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', GradientBoostingRegressor(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=5,
            random_state=RANDOM_STATE
        ))
    ])
}

print(" 3 modelos definidos con pipelines completos")
print("   Cada pipeline incluye: preprocesamiento + modelo")

# ═══ ENTRENAMIENTO Y EVALUACIÓN ═══
print("\n" + "-"*80)
print(" ENTRENAMIENTO Y EVALUACIÓN")
print("-"*80)

results = []

for model_name, pipeline in models.items():
    print(f"\n{'='*60}")
    print(f" {model_name}")
    print('='*60)
    
    # ═══ VALIDACIÓN CRUZADA ═══
    print("    Validación cruzada (5 folds)...")
    cv_scores_rmse = -cross_val_score(
        pipeline, X_train, y_train,
        cv=5,
        scoring='neg_root_mean_squared_error',
        n_jobs=-1
    )
    cv_scores_r2 = cross_val_score(
        pipeline, X_train, y_train,
        cv=5,
        scoring='r2',
        n_jobs=-1
    )
    
    cv_rmse_mean = cv_scores_rmse.mean()
    cv_rmse_std = cv_scores_rmse.std()
    cv_r2_mean = cv_scores_r2.mean()
    
    print(f"    CV RMSE: {cv_rmse_mean:,.0f} ± {cv_rmse_std:,.0f}")
    print(f"    CV R²: {cv_r2_mean:.4f}")
    
    # ═══ ENTRENAMIENTO EN TRAIN COMPLETO ═══
    print("    Entrenando en train completo...")
    pipeline.fit(X_train, y_train)
    
    # ═══ PREDICCIONES ═══
    y_train_pred = pipeline.predict(X_train)
    y_test_pred = pipeline.predict(X_test)
    
    # ═══ MÉTRICAS TRAIN ═══
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    
    # ═══ MÉTRICAS TEST ═══
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    # ═══ DIAGNÓSTICO DE OVERFITTING ═══
    overfitting_ratio = test_rmse / train_rmse
    
    print(f"\n    Métricas Train:")
    print(f"      • RMSE: {train_rmse:,.0f}")
    print(f"      • MAE: {train_mae:,.0f}")
    print(f"      • R²: {train_r2:.4f}")
    
    print(f"\n    Métricas Test:")
    print(f"      • RMSE: {test_rmse:,.0f}")
    print(f"      • MAE: {test_mae:,.0f}")
    print(f"      • R²: {test_r2:.4f}")
    
    print(f"\n    Diagnóstico:")
    print(f"      • Ratio Test/Train RMSE: {overfitting_ratio:.2f}x")
    if overfitting_ratio < 1.2:
        print(f"      ✅ Excelente generalización")
    elif overfitting_ratio < 1.5:
        print(f"      ✓ Buena generalización")
    elif overfitting_ratio < 2.5:
        print(f"      ⚠️ Overfitting moderado")
    else:
        print(f"      🚨 Overfitting severo")
    
    # Guardar resultados
    results.append({
        'Modelo': model_name,
        'CV_RMSE': cv_rmse_mean,
        'CV_R²': cv_r2_mean,
        'Train_RMSE': train_rmse,
        'Train_R²': train_r2,
        'Test_RMSE': test_rmse,
        'Test_MAE': test_mae,
        'Test_R²': test_r2,
        'Overfitting_Ratio': overfitting_ratio,
        'pipeline': pipeline
    })

# ═══ TABLA COMPARATIVA ═══
print("\n" + "="*80)
print(" COMPARACIÓN DE MODELOS")
print("="*80)

comparison_df = pd.DataFrame(results)
comparison_df_sorted = comparison_df.sort_values('Test_RMSE')

print("\n Tabla comparativa (ordenada por Test RMSE):")
display_cols = ['Modelo', 'CV_RMSE', 'Test_RMSE', 'Test_MAE', 'Test_R²', 'Overfitting_Ratio']
print(comparison_df_sorted[display_cols].to_string(index=False))

# ═══ SELECCIÓN DEL MEJOR MODELO ═══
best_model_idx = comparison_df_sorted.index[0]
best_model_name = comparison_df_sorted.iloc[0]['Modelo']
best_pipeline = comparison_df_sorted.iloc[0]['pipeline']
best_test_rmse = comparison_df_sorted.iloc[0]['Test_RMSE']
best_test_r2 = comparison_df_sorted.iloc[0]['Test_R²']

print(f"\n{'='*60}")
print(f"🏆 MEJOR MODELO: {best_model_name}")
print('='*60)
print(f"   • Test RMSE: {best_test_rmse:,.0f}")
print(f"   • Test R²: {best_test_r2:.4f} ({best_test_r2*100:.1f}% varianza explicada)")



print(" Comparación completada - Mejor modelo seleccionado")


 DEFINICIÓN DE MODELOS
 3 modelos definidos con pipelines completos
   Cada pipeline incluye: preprocesamiento + modelo

--------------------------------------------------------------------------------
 ENTRENAMIENTO Y EVALUACIÓN
--------------------------------------------------------------------------------

 Ridge Regression
    Validación cruzada (5 folds)...
    CV RMSE: 10,517 ± 610
    CV R²: 0.5162
    Entrenando en train completo...

    Métricas Train:
      • RMSE: 10,135
      • MAE: 7,109
      • R²: 0.5716

    Métricas Test:
      • RMSE: 37,167
      • MAE: 34,307
      • R²: -4.2007

    Diagnóstico:
      • Ratio Test/Train RMSE: 3.67x
      🚨 Overfitting severo

 Random Forest
    Validación cruzada (5 folds)...
    CV RMSE: 8,070 ± 2,565
    CV R²: 0.6600
    Entrenando en train completo...

    Métricas Train:
      • RMSE: 5,084
      • MAE: 1,969
      • R²: 0.8922

    Métricas Test:
      • RMSE: 7,705
      • MAE: 3,912
      • R²: 0.7765

    Diagnóstico:
  

In [6]:
# %%
"""
════════════════════════════════════════════════════════════════════════════════
PASO 4: OPTIMIZACIÓN DE HIPERPARÁMETROS
════════════════════════════════════════════════════════════════════════════════
Objetivo: Ajustar los hiperparámetros del mejor modelo usando RandomizedSearchCV

Técnica: RandomizedSearchCV
   • Explora ALEATORIAMENTE el espacio de hiperparámetros
   • Más rápido que GridSearchCV exhaustivo 
   
Validación: 5-fold cross-validation
"""

from sklearn.model_selection import RandomizedSearchCV
import time
from datetime import datetime

# ═══════════════════════════════════════════════════════════════════════════
# DEFINICIÓN DEL ESPACIO DE BÚSQUEDA
# ═══════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print(f" OPTIMIZACIÓN: {best_model_name}")
print("="*80)

# Espacio de búsqueda según el modelo
if 'Forest' in best_model_name:
    print("\n Espacio de búsqueda para Random Forest:")
    param_distributions = {
        'regressor__n_estimators': [100, 150, 200, 250, 300],
        'regressor__max_depth': [15, 20, 25, 30, None],
        'regressor__min_samples_split': [2, 4, 8, 12],
        'regressor__min_samples_leaf': [1, 2, 3, 4],
        'regressor__max_features': ['sqrt', 0.6, 0.7, 0.8, 1.0],
        'regressor__max_samples': [0.7, 0.8, 0.9, 1.0]
    }
    n_iter = 40
    
elif 'Gradient' in best_model_name or 'Boosting' in best_model_name:
    print("\n📋 Espacio de búsqueda para Gradient Boosting:")
    print("   Estrategia: Mayor complejidad (modelo ya generaliza bien)")
    
    param_distributions = {
        'regressor__n_estimators': [100, 150, 200, 250, 300, 350],
        'regressor__learning_rate': [0.05, 0.08, 0.1, 0.12, 0.15],
        'regressor__max_depth': [3, 4, 5, 6, 7],
        'regressor__min_samples_split': [2, 4, 8, 12, 16],
        'regressor__min_samples_leaf': [1, 2, 3, 4, 6],
        'regressor__subsample': [0.7, 0.8, 0.9, 1.0],
        'regressor__max_features': ['sqrt', 0.5, 0.7, 0.8, 1.0]
    }
    n_iter = 50
    
else:  # Ridge
    print("\n Espacio de búsqueda para Ridge:")
    param_distributions = {
        'regressor__alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0],
        'regressor__solver': ['auto', 'svd', 'cholesky', 'lsqr', 'saga'],
        'regressor__max_iter': [1000, 2000, 5000]
    }
    n_iter = 30

# Mostrar parámetros
print(f"\n Hiperparámetros a explorar:")
for param, values in param_distributions.items():
    param_name = param.replace('regressor__', '')
    print(f"   • {param_name}: {len(values)} valores")

# Calcular combinaciones posibles
n_combinations = np.prod([len(v) for v in param_distributions.values()])
print(f"\n Combinaciones posibles: {n_combinations:,}")
print(f"   Iteraciones a realizar: {n_iter}")
print(f"   Exploración: {n_iter/n_combinations*100:.2f}% del espacio")

# ═══════════════════════════════════════════════════════════════════════════
# EJECUCIÓN DE RANDOMIZEDSEARCHCV
# ═══════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print(" EJECUTANDO RANDOMIZEDSEARCHCV")
print("="*80)

print(f"\n Configuración:")
print(f"   • Método: RandomizedSearchCV (aleatorio)")
print(f"   • Iteraciones: {n_iter}")
print(f"   • CV folds: 5")
print(f"   • Total entrenamientos: {n_iter * 5}")
print(f"   • Paralelización: Todos los cores (n_jobs=-1)")



start_time = time.time()

random_search = RandomizedSearchCV(
    estimator=best_pipeline,
    param_distributions=param_distributions,
    n_iter=n_iter,
    cv=5,
    scoring='neg_root_mean_squared_error',
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=2,  # Mostrar progreso
    return_train_score=True
)

print("\n Buscando mejores hiperparámetros...")
print("   (verbose=2 mostrará progreso de cada fold)\n")

random_search.fit(X_train, y_train)

elapsed_time = time.time() - start_time
elapsed_minutes = elapsed_time / 60

print(f"\n{'='*80}")
print(f" BÚSQUEDA COMPLETADA")
print(f"{'='*80}")

# ═══════════════════════════════════════════════════════════════════════════
# MEJORES HIPERPARÁMETROS ENCONTRADOS
# ═══════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print(" RESULTADOS DE LA OPTIMIZACIÓN")
print("="*80)

print("\n Mejores hiperparámetros encontrados:")
for param, value in random_search.best_params_.items():
    param_name = param.replace('regressor__', '')
    print(f"   • {param_name}: {value}")

# ═══════════════════════════════════════════════════════════════════════════
# EVALUACIÓN DEL MODELO OPTIMIZADO
# ═══════════════════════════════════════════════════════════════════════════

best_cv_score = -random_search.best_score_

print(f"\n Validación Cruzada:")
print(f"   • CV RMSE optimizado: {best_cv_score:,.0f}")

# Predicciones con modelo optimizado
y_train_pred_opt = random_search.best_estimator_.predict(X_train)
y_test_pred_opt = random_search.best_estimator_.predict(X_test)

# Métricas
train_rmse_opt = np.sqrt(mean_squared_error(y_train, y_train_pred_opt))
train_r2_opt = r2_score(y_train, y_train_pred_opt)

test_rmse_opt = np.sqrt(mean_squared_error(y_test, y_test_pred_opt))
test_mae_opt = mean_absolute_error(y_test, y_test_pred_opt)
test_r2_opt = r2_score(y_test, y_test_pred_opt)

overfitting_ratio_opt = test_rmse_opt / train_rmse_opt

print(f"\n Test Set (modelo optimizado):")
print(f"   • Test RMSE: {test_rmse_opt:,.0f}")
print(f"   • Test MAE: {test_mae_opt:,.0f}")
print(f"   • Test R²: {test_r2_opt:.4f}")
print(f"   • Train RMSE: {train_rmse_opt:,.0f}")
print(f"   • Overfitting ratio: {overfitting_ratio_opt:.2f}x")

# ═══════════════════════════════════════════════════════════════════════════
# COMPARACIÓN: ORIGINAL VS OPTIMIZADO
# ═══════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print(" COMPARACIÓN: ORIGINAL VS OPTIMIZADO")
print("="*80)

# ───────────────────────────────────────────────────────────────────────────
# MÉTRICAS DEL MODELO ORIGINAL (de CELDA 4)
# ───────────────────────────────────────────────────────────────────────────

print("\n Calculando métricas del modelo original...")

# Predicciones con modelo original
y_train_pred_original = best_pipeline.predict(X_train)
y_test_pred_original = best_pipeline.predict(X_test)

# Métricas originales
original_train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred_original))
original_test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred_original))
original_test_mae = mean_absolute_error(y_test, y_test_pred_original)
original_test_r2 = r2_score(y_test, y_test_pred_original)
original_overfitting = original_test_rmse / original_train_rmse

print(f"\n Modelo Original:")
print(f"   • Train RMSE: {original_train_rmse:,.0f}")
print(f"   • Test RMSE: {original_test_rmse:,.0f}")
print(f"   • Test MAE: {original_test_mae:,.0f}")
print(f"   • Test R²: {original_test_r2:.4f}")
print(f"   • Overfitting ratio: {original_overfitting:.2f}x")

# ───────────────────────────────────────────────────────────────────────────
# COMPARACIÓN
# ───────────────────────────────────────────────────────────────────────────

# Calcular mejoras
mejora_rmse = ((original_test_rmse - test_rmse_opt) / original_test_rmse) * 100
mejora_r2 = ((test_r2_opt - original_test_r2) / original_test_r2) * 100
cambio_overfitting = ((overfitting_ratio_opt - original_overfitting) / original_overfitting) * 100

print(f"\n Tabla Comparativa:")
print(f"\n{'Métrica':<25} {'Original':<15} {'Optimizado':<15} {'Cambio':<10}")
print("─"*65)
print(f"{'Train RMSE':<25} {original_train_rmse:<15,.0f} {train_rmse_opt:<15,.0f} {((train_rmse_opt-original_train_rmse)/original_train_rmse*100):>9.2f}%")
print(f"{'Test RMSE':<25} {original_test_rmse:<15,.0f} {test_rmse_opt:<15,.0f} {mejora_rmse:>9.2f}%")
print(f"{'Test MAE':<25} {original_test_mae:<15,.0f} {test_mae_opt:<15,.0f} {((test_mae_opt-original_test_mae)/original_test_mae*100):>9.2f}%")
print(f"{'Test R²':<25} {original_test_r2:<15.4f} {test_r2_opt:<15.4f} {mejora_r2:>9.2f}%")
print(f"{'Overfitting Ratio':<25} {original_overfitting:<15.2f}x {overfitting_ratio_opt:<15.2f}x {cambio_overfitting:>9.2f}%")

# ───────────────────────────────────────────────────────────────────────────
# ANÁLISIS DETALLADO
# ───────────────────────────────────────────────────────────────────────────

print(f"\n Análisis de mejoras:")

if mejora_rmse > 0:
    print(f"    RMSE mejoró: {mejora_rmse:.2f}%")
    print(f"      → Error reducido en {original_test_rmse - test_rmse_opt:,.0f} turistas")
elif mejora_rmse < 0:
    print(f"    RMSE empeoró: {mejora_rmse:.2f}%")
    print(f"      → Error aumentó en {test_rmse_opt - original_test_rmse:,.0f} turistas")
else:
    print(f"   ➖ RMSE sin cambios")

if mejora_r2 > 0:
    print(f"    R² mejoró: {mejora_r2:.2f}%")
    print(f"      → Explica {test_r2_opt:.2%} de la varianza (antes {original_test_r2:.2%})")
elif mejora_r2 < 0:
    print(f"    R² empeoró: {mejora_r2:.2f}%")
else:
    print(f"   ➖ R² sin cambios")

if cambio_overfitting < 0:
    print(f"    Overfitting reducido: {abs(cambio_overfitting):.2f}%")
    print(f"      → Mejor generalización ({overfitting_ratio_opt:.2f}x vs {original_overfitting:.2f}x)")
elif cambio_overfitting > 0:
    print(f"    Overfitting aumentó: {cambio_overfitting:.2f}%")
    print(f"      → Ratio: {overfitting_ratio_opt:.2f}x (antes {original_overfitting:.2f}x)")
else:
    print(f"   ➖ Overfitting sin cambios")

# ═══════════════════════════════════════════════════════════════════════════
# DECISIÓN FINAL
# ═══════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print(" DECISIÓN FINAL")
print("="*80)

# Criterios de decisión
mejoro_rmse = test_rmse_opt < original_test_rmse
mejoro_r2 = test_r2_opt > original_test_r2
overfitting_aceptable = overfitting_ratio_opt < 2.0
mejoro_significativamente = mejora_rmse > 2.0  # Mejora mínima 2%

if mejoro_rmse and overfitting_aceptable:
    print(f"\n USAR MODELO OPTIMIZADO")
    print(f"\n   Razones:")
    print(f"   • Test RMSE mejoró: {mejora_rmse:.2f}%")
    print(f"   • Reducción de error: {original_test_rmse - test_rmse_opt:,.0f} turistas")
    print(f"   • Overfitting controlado: {overfitting_ratio_opt:.2f}x")
    
    if mejoro_r2:
        print(f"   • R² también mejoró: {mejora_r2:.2f}%")
    
    if mejoro_significativamente:
        print(f"   • Mejora SIGNIFICATIVA (>{2}%)")
    else:
        print(f"   • Mejora marginal pero positiva")
    
    # Actualizar variables finales
    final_pipeline = random_search.best_estimator_
    test_rmse_final = test_rmse_opt
    test_mae_final = test_mae_opt
    test_r2_final = test_r2_opt
    train_rmse_final = train_rmse_opt
    train_r2_final = train_r2_opt
    overfitting_ratio_final = overfitting_ratio_opt
    
    print(f"\n    Modelo optimizado guardado como 'final_pipeline'")
    
elif not mejoro_rmse and mejoro_r2 and overfitting_aceptable:
    print(f"\n DECISIÓN MIXTA")
    print(f"\n   • RMSE empeoró: {mejora_rmse:.2f}%")
    print(f"   • Pero R² mejoró: {mejora_r2:.2f}%")
    print(f"   • Overfitting: {overfitting_ratio_opt:.2f}x")
    print(f"\n   Recomendación: Mantener ORIGINAL (RMSE es métrica principal)")
    
    final_pipeline = best_pipeline
    test_rmse_final = original_test_rmse
    test_mae_final = original_test_mae
    test_r2_final = original_test_r2
    train_rmse_final = original_train_rmse
    overfitting_ratio_final = original_overfitting
    
else:
    print(f"\n MANTENER MODELO ORIGINAL")
    print(f"\n   Razones:")
    
    if not mejoro_rmse:
        print(f"   • RMSE no mejoró (cambio: {mejora_rmse:.2f}%)")
        print(f"   • Diferencia: {test_rmse_opt - original_test_rmse:,.0f} turistas")
    
    if not overfitting_aceptable:
        print(f"   • Overfitting demasiado alto: {overfitting_ratio_opt:.2f}x")
        print(f"   • Umbral recomendado: < 2.0x")
    
    if not mejoro_r2:
        print(f"   • R² no mejoró (cambio: {mejora_r2:.2f}%)")
    
    print(f"\n   El modelo original tiene mejor balance performance/generalización")
    
    # Mantener variables originales
    final_pipeline = best_pipeline
    test_rmse_final = original_test_rmse
    test_mae_final = original_test_mae
    test_r2_final = original_test_r2
    train_rmse_final = original_train_rmse
    overfitting_ratio_final = original_overfitting
    
    print(f"\n    Modelo original mantenido como 'final_pipeline'")

# ───────────────────────────────────────────────────────────────────────────
# RESUMEN FINAL
# ───────────────────────────────────────────────────────────────────────────

print("\n" + "="*80)
print(" RESUMEN FINAL")
print("="*80)

print(f"\n Modelo seleccionado: {best_model_name}")
print(f"\n Métricas finales:")
print(f"   • Test RMSE: {test_rmse_final:,.0f} turistas")
print(f"   • Test MAE: {test_mae_final:,.0f} turistas")
print(f"   • Test R²: {test_r2_final:.4f} ({test_r2_final*100:.2f}% de varianza explicada)")
print(f"   • Train RMSE: {train_rmse_final:,.0f} turistas")
print(f"   • Overfitting ratio: {overfitting_ratio_final:.2f}x")

if overfitting_ratio_final < 1.5:
    print(f"\n    Excelente generalización (ratio < 1.5x)")
elif overfitting_ratio_final < 2.0:
    print(f"\n   ✓ Buena generalización (ratio < 2.0x)")
else:
    print(f"\n    Overfitting moderado (ratio > 2.0x)")

print("\n" + "="*80)
print(" OPTIMIZACIÓN COMPLETADA")
print("="*80)


 OPTIMIZACIÓN: Gradient Boosting

📋 Espacio de búsqueda para Gradient Boosting:
   Estrategia: Mayor complejidad (modelo ya generaliza bien)

 Hiperparámetros a explorar:
   • n_estimators: 6 valores
   • learning_rate: 5 valores
   • max_depth: 5 valores
   • min_samples_split: 5 valores
   • min_samples_leaf: 5 valores
   • subsample: 4 valores
   • max_features: 5 valores

 Combinaciones posibles: 75,000
   Iteraciones a realizar: 50
   Exploración: 0.07% del espacio

 EJECUTANDO RANDOMIZEDSEARCHCV

 Configuración:
   • Método: RandomizedSearchCV (aleatorio)
   • Iteraciones: 50
   • CV folds: 5
   • Total entrenamientos: 250
   • Paralelización: Todos los cores (n_jobs=-1)

 Buscando mejores hiperparámetros...
   (verbose=2 mostrará progreso de cada fold)

Fitting 5 folds for each of 50 candidates, totalling 250 fits

 BÚSQUEDA COMPLETADA

 RESULTADOS DE LA OPTIMIZACIÓN

 Mejores hiperparámetros encontrados:
   • subsample: 1.0
   • n_estimators: 200
   • min_samples_split: 12
   

In [7]:
# %%
"""
════════════════════════════════════════════════════════════════════════════════
PASO 5: EVALUACIÓN FINAL DEL MODELO
════════════════════════════════════════════════════════════════════════════════
Objetivo: Analizar el rendimiento del modelo final en profundidad
   • Métricas finales en test
   • Análisis de residuos
   • Feature importance (si aplica)
   • Diagnóstico final de overfitting/underfitting
"""

# ═══ PREDICCIONES FINALES ═══
print("\n" + "="*80)
print("EVALUACIÓN FINAL DEL MODELO")
print("="*80)

y_train_pred_final = final_pipeline.predict(X_train)
y_test_pred_final = final_pipeline.predict(X_test)

# ═══ MÉTRICAS FINALES ═══
print("\n" + "-"*80)
print("MÉTRICAS FINALES")
print("-"*80)

# Train
train_rmse_final = np.sqrt(mean_squared_error(y_train, y_train_pred_final))
train_mae_final = mean_absolute_error(y_train, y_train_pred_final)
train_r2_final = r2_score(y_train, y_train_pred_final)

# Test
test_rmse_final = np.sqrt(mean_squared_error(y_test, y_test_pred_final))
test_mae_final = mean_absolute_error(y_test, y_test_pred_final)
test_r2_final = r2_score(y_test, y_test_pred_final)

print(f"\n{'Conjunto':<10} {'RMSE':<15} {'MAE':<15} {'R²':<10}")
print("-" * 55)
print(f"{'Train':<10} {train_rmse_final:>14,.0f} {train_mae_final:>14,.0f} {train_r2_final:>9.4f}")
print(f"{'Test':<10} {test_rmse_final:>14,.0f} {test_mae_final:>14,.0f} {test_r2_final:>9.4f}")

# ═══ DIAGNÓSTICO FINAL ═══
print("\n" + "-"*80)
print("DIAGNÓSTICO FINAL")
print("-"*80)

overfitting_ratio_final = test_rmse_final / train_rmse_final

print(f"\nAnálisis de Overfitting/Underfitting:")
print(f"   • Ratio Test/Train RMSE: {overfitting_ratio_final:.2f}x")

if overfitting_ratio_final < 1.1:
    diagnosis = " EXCELENTE - Sin overfitting"
    explanation = "El modelo generaliza perfectamente"
elif overfitting_ratio_final < 1.3:
    diagnosis = " MUY BUENO - Overfitting mínimo"
    explanation = "El modelo generaliza muy bien"
elif overfitting_ratio_final < 1.5:
    diagnosis = "✓ BUENO - Overfitting leve"
    explanation = "El modelo generaliza bien, con margen de mejora"
elif overfitting_ratio_final < 2.5:
    diagnosis = " MODERADO - Overfitting moderado"
    explanation = "El modelo memoriza algo el train, pero aún generaliza"
else:
    diagnosis = " ALTO - Overfitting severo"
    explanation = "El modelo memoriza demasiado el train"

print(f"   • {diagnosis}")
print(f"   • {explanation}")

# Interpretación de R²
print(f"\nInterpretación de R²:")
print(f"   • R² Test = {test_r2_final:.4f}")
print(f"   → El modelo explica {test_r2_final*100:.1f}% de la variabilidad en turistas")

if test_r2_final >= 0.9:
    r2_interp = "EXCELENTE"
elif test_r2_final >= 0.7:
    r2_interp = "MUY BUENO"
elif test_r2_final >= 0.5:
    r2_interp = "BUENO"
elif test_r2_final >= 0.3:
    r2_interp = "REGULAR"
else:
    r2_interp = "BAJO"

print(f"   → Clasificación: {r2_interp}")

# ═══ ANÁLISIS DE RESIDUOS ═══
print("\n" + "-"*80)
print("ANÁLISIS DE RESIDUOS")
print("-"*80)

residuals_test = y_test - y_test_pred_final

print(f"\nEstadísticas de residuos en test:")
print(f"   • Media: {residuals_test.mean():,.0f}")
print(f"   • Mediana: {residuals_test.median():,.0f}")
print(f"   • Desv. Estándar: {residuals_test.std():,.0f}")
print(f"   • Rango: {residuals_test.min():,.0f} a {residuals_test.max():,.0f}")


plt.suptitle(f'Evaluación Final - {best_model_name}', fontsize=14, fontweight='bold')
plt.show()



print("\n Evaluación final completada")


EVALUACIÓN FINAL DEL MODELO

--------------------------------------------------------------------------------
MÉTRICAS FINALES
--------------------------------------------------------------------------------

Conjunto   RMSE            MAE             R²        
-------------------------------------------------------
Train               5,010          2,054    0.8953
Test                7,397          3,929    0.7940

--------------------------------------------------------------------------------
DIAGNÓSTICO FINAL
--------------------------------------------------------------------------------

Análisis de Overfitting/Underfitting:
   • Ratio Test/Train RMSE: 1.48x
   • ✓ BUENO - Overfitting leve
   • El modelo generaliza bien, con margen de mejora

Interpretación de R²:
   • R² Test = 0.7940
   → El modelo explica 79.4% de la variabilidad en turistas
   → Clasificación: MUY BUENO

--------------------------------------------------------------------------------
ANÁLISIS DE RESIDUOS
-

<Figure size 640x480 with 0 Axes>


 Evaluación final completada


In [8]:
# %%
"""
════════════════════════════════════════════════════════════════════════════════
PASO 6: CONCLUSIONES Y RECOMENDACIONES
════════════════════════════════════════════════════════════════════════════════
Resumen del proyecto y aprendizajes clave

"""

print("\n" + "="*80)
print(" CONCLUSIONES DEL PROYECTO")
print("="*80)

# ═══════════════════════════════════════════════════════════════════════════
# RESUMEN EJECUTIVO
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-"*80)
print(" RESUMEN EJECUTIVO")
print("-"*80)

print(f"""
OBJETIVO:
   Predecir la demanda turística mensual en Mendoza para optimizar 
   planificación hotelera y gestión de recursos.

METODOLOGÍA:
    Pipeline completo de Scikit-learn (preprocesamiento + modelo)
    Split temporal 80/20 (respeta orden cronológico)
    Validación cruzada 5-fold para evaluación robusta
    3 modelos comparados (Ridge, Random Forest, Gradient Boosting)
    Optimización de hiperparámetros con RandomizedSearchCV

MODELO FINAL:
    {best_model_name}
   
MÉTRICAS FINALES:
   • Test RMSE: {test_rmse_final:,.0f} turistas
   • Test MAE: {test_mae_final:,.0f} turistas  
   • Test R²: {test_r2_final:.4f} ({test_r2_final*100:.1f}% varianza explicada)
   • Ratio Overfitting: {overfitting_ratio_final:.2f}x
   • Diagnóstico: {diagnosis}
""")

# ═══════════════════════════════════════════════════════════════════════════
# INTERPRETACIÓN DE RESULTADOS
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-"*80)
print(" INTERPRETACIÓN DE RESULTADOS")
print("-"*80)

# Calcular error relativo
error_relativo = (test_rmse_final / y_test.mean()) * 100

print(f"""
¿QUÉ SIGNIFICAN ESTAS MÉTRICAS?

1️ RMSE = {test_rmse_final:,.0f} turistas:
   → En promedio, el modelo se equivoca por {test_rmse_final:,.0f} turistas
   → Para un promedio de {y_test.mean():,.0f} turistas, esto representa
     un error del {error_relativo:.1f}%

2️ R² = {test_r2_final:.4f}:
   → El modelo explica {test_r2_final*100:.1f}% de la variabilidad
   → Clasificación: {r2_interp}
   → El {(1-test_r2_final)*100:.1f}% restante se debe a:
      • Factores no incluidos (clima, eventos, economía global)
      • Ruido inherente en los datos
      • Variabilidad aleatoria

3️ Overfitting ratio = {overfitting_ratio_final:.2f}x:
   → {explanation}
""")

# Interpretación adicional del diagnóstico
if 'SEVERO' in diagnosis or 'MODERADO' in diagnosis:
    print(f"    Nota sobre el overfitting:")
    print(f"      Este overfitting puede deberse a:")
    print(f"      • Dataset con período excepcional (pandemia 2020-2021)")
    print(f"      • Drift temporal entre train y test")
    print(f"      • Es ESPERADO en datos con alta variabilidad temporal")
elif 'LEVE' in diagnosis:
    print(f"   ✓ El overfitting es leve y aceptable")
else:
    print(f"    Excelente balance entre train y test")

# ═══════════════════════════════════════════════════════════════════════════
# FORTALEZAS DEL MODELO
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-"*80)
print(" FORTALEZAS DEL MODELO")
print("-"*80)

print(f"""
1. Pipeline Completo y Reproducible:
   ✓ Integra preprocesamiento y modelado en un solo objeto
   ✓ Previene data leakage (parámetros aprendidos solo del train)
   ✓ Garantiza consistencia entre train y test

2. Validación Rigurosa:
   ✓ Validación cruzada 5-fold con TimeSeriesSplit
   ✓ Split temporal (simula forecasting real)
   ✓ Evaluación en datos no vistos (test set)
   ✓ Coherencia entre CV y test verificada

3. Capacidad Predictiva:
   ✓ R² = {test_r2_final:.4f} indica {'excelente' if test_r2_final >= 0.7 else 'buena'} capacidad explicativa
   ✓ Supera a alternativas más simples (Ridge)
   ✓ Error relativo del {error_relativo:.1f}% es razonable para el contexto

4. Interpretabilidad:
   ✓ Feature importance disponible (top features identificados)
   ✓ Residuos analizados (comportamiento entendido)
   ✓ Decisiones documentadas y justificadas
""")

# ═══════════════════════════════════════════════════════════════════════════
# LIMITACIONES
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-"*80)
print(" LIMITACIONES Y CONSIDERACIONES")
print("-"*80)

print("""
1. Registros con Cero Eliminados:
    Se eliminaron 389 registros (13.5%) con 0 turistas
   → Correspondientes al período de pandemia (2020-2021)

2. Horizonte Temporal:
    Predicción mensual agregada
   → No captura variación diaría
   → Útil para planificación estratégica (2-6 meses)
   → Menos útil para decisiones operativas diarias
""")

print(f"\n4. Error Relativo:")
print(f"   • RMSE relativo: {error_relativo:.1f}%")
if error_relativo < 30:
    print(f"    Excelente para planificación estratégica")
elif error_relativo < 50:
    print(f"   ✓ Aceptable para planificación estratégica")
    print(f"    Puede ser alto para decisiones operativas críticas")
else:
    print(f"    Alto - Usar con precaución para decisiones críticas")


# ═══════════════════════════════════════════════════════════════════════════
# GUÍA DE IMPLEMENTACIÓN EN PRODUCCIÓN
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "-"*80)
print(" GUÍA DE IMPLEMENTACIÓN EN PRODUCCIÓN")
print("-"*80)

print("""
PASO 1: GUARDAR EL MODELO
""")

# Mostrar código sin ejecutarlo
codigo_guardar = '''import joblib

# Guardar pipeline completo
joblib.dump(final_pipeline, 'modelo_turismo_mendoza_v1.pkl')

# Guardar también el preprocessor separado (opcional)
joblib.dump(preprocessor, 'preprocessor_turismo_mendoza.pkl')

print(" Modelo guardado en disco")
'''

print(codigo_guardar)

print("""
PASO 2: CARGAR Y USAR EL MODELO
""")

codigo_cargar = '''import joblib
import pandas as pd

# Cargar modelo
modelo = joblib.load('modelo_turismo_mendoza_v1.pkl')

# Preparar datos nuevos (mismo formato que training)
nuevos_datos = pd.DataFrame({
    'precio_promedio_usd': [850, 920],
    'mes': [7, 8],
    'año': [2025, 2025],
    'pais_origen': ['Chile', 'Brasil'],
    'punto_entrada': ['Aeropuerto Buenos Aires', 'Aeropuerto Córdoba'],
    # ... todas las demás features ...
})

# Predecir
predicciones = modelo.predict(nuevos_datos)

print(f"Predicción mes 7/2025: {predicciones[0]:,.0f} turistas")
print(f"Predicción mes 8/2025: {predicciones[1]:,.0f} turistas")
'''

print(codigo_cargar)

print("""
PASO 3: MONITOREO DE PERFORMANCE
""")

codigo_monitoreo = '''import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# Comparar predicciones vs reales cada mes
predicciones_mes = modelo.predict(X_real)
turistas_reales = y_real

# Calcular métricas
rmse_mes = np.sqrt(mean_squared_error(turistas_reales, predicciones_mes))
r2_mes = r2_score(turistas_reales, predicciones_mes)

print(f"RMSE este mes: {rmse_mes:,.0f}")
print(f"R² este mes: {r2_mes:.3f}")

# Alertas
if rmse_mes > test_rmse_final * 1.5:
    print(" ALERTA: Performance degradada - Considerar reentrenamiento")
'''

print(codigo_monitoreo)

# ═══════════════════════════════════════════════════════════════════════════
# RESUMEN FINAL
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "="*80)
print(" RESUMEN FINAL DEL PROYECTO")
print("="*80)

print(f"""
 PROYECTO COMPLETADO EXITOSAMENTE

MODELO SELECCIONADO:
    {best_model_name}
   • Test RMSE: {test_rmse_final:,.0f} ({error_relativo:.1f}% error relativo)
   • Test R²: {test_r2_final:.4f} ({test_r2_final*100:.1f}% varianza explicada)
   • Clasificación: {r2_interp}

METODOLOGÍA APLICADA:
   ✓ Pipeline completo de Scikit-learn
   ✓ 3 modelos comparados rigurosamente
   ✓ Validación cruzada temporal (5-fold)
   ✓ Optimización de hiperparámetros
   ✓ Eliminación de registros con 0 turistas

DECISIONES TOMADAS:
   1. Eliminar 389 registros con 0 turistas (pandemia)
   
   2. Split temporal 80/20
      → Simula forecasting real
   
   3. StandardScaler + OneHotEncoder
      → Pipeline estándar de Scikit-learn
   
   4. {best_model_name} seleccionado
      → Mejor balance performance/generalización

""")



 CONCLUSIONES DEL PROYECTO

--------------------------------------------------------------------------------
 RESUMEN EJECUTIVO
--------------------------------------------------------------------------------

OBJETIVO:
   Predecir la demanda turística mensual en Mendoza para optimizar 
   planificación hotelera y gestión de recursos.

METODOLOGÍA:
    Pipeline completo de Scikit-learn (preprocesamiento + modelo)
    Split temporal 80/20 (respeta orden cronológico)
    Validación cruzada 5-fold para evaluación robusta
    3 modelos comparados (Ridge, Random Forest, Gradient Boosting)
    Optimización de hiperparámetros con RandomizedSearchCV

MODELO FINAL:
    Gradient Boosting

MÉTRICAS FINALES:
   • Test RMSE: 7,397 turistas
   • Test MAE: 3,929 turistas  
   • Test R²: 0.7940 (79.4% varianza explicada)
   • Ratio Overfitting: 1.48x
   • Diagnóstico: ✓ BUENO - Overfitting leve


--------------------------------------------------------------------------------
 INTERPRETACIÓN DE RESUL

In [10]:
# %%
"""
════════════════════════════════════════════════════════════════════════════════
CELDA 9: EXPORTACIÓN DEL MODELO PARA PRODUCCIÓN (VERSIÓN CORREGIDA)
════════════════════════════════════════════════════════════════════════════════
Objetivo: Serializar el modelo entrenado con categorías limpias
Versión: 2.0 - Sin dependencias de variables faltantes
Autor: Juliccc
Fecha: 2025-11-05
"""

import joblib
import json
from datetime import datetime
import os
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

print("\n" + "="*80)
print("💾 EXPORTACIÓN DEL MODELO PARA PRODUCCIÓN (v2.0)")
print("="*80)

# ═══════════════════════════════════════════════════════════════════════════
# CREAR DIRECTORIO DE MODELOS
# ═══════════════════════════════════════════════════════════════════════════
print("\n📁 Creando estructura de directorios...")

if not os.path.exists('models'):
    os.makedirs('models')
    print("✅ Directorio 'models/' creado")
else:
    print("✅ Directorio 'models/' ya existe")

# ═══════════════════════════════════════════════════════════════════════════
# GUARDAR PIPELINE COMPLETO
# ═══════════════════════════════════════════════════════════════════════════
print("\n💾 Guardando pipeline del modelo...")

model_filename = 'models/modelo_turismo_mendoza_final.pkl'
joblib.dump(final_pipeline, model_filename, compress=3)

print(f"✅ Modelo guardado en: {model_filename}")
print(f"   Tamaño: {os.path.getsize(model_filename) / 1024:.2f} KB")

# ═══════════════════════════════════════════════════════════════════════════
# EXTRAER CATEGORÍAS ÚNICAS LIMPIAS
# ═══════════════════════════════════════════════════════════════════════════
print("\n🔍 Extrayendo categorías únicas del dataset limpio...")

categorias_limpias = {}

for col in categorical_features:
    if col in df.columns:
        valores_unicos = sorted(df[col].dropna().unique().tolist())
        categorias_limpias[col] = valores_unicos
        
        print(f"\n   📊 {col}: {len(valores_unicos)} categorías")
        for i, cat in enumerate(valores_unicos, 1):
            count = (df[col] == cat).sum()
            pct = count / len(df) * 100
            print(f"      {i}. {cat:<45} ({count:>5} registros, {pct:>5.2f}%)")

# ═══════════════════════════════════════════════════════════════════════════
# CALCULAR MÉTRICAS FINALES
# ═══════════════════════════════════════════════════════════════════════════
print("\n📊 Calculando métricas finales...")

# Predicciones
y_pred_train = final_pipeline.predict(X_train)
y_pred_test = final_pipeline.predict(X_test)

# Métricas train
train_rmse_final = np.sqrt(mean_squared_error(y_train, y_pred_train))
train_r2_final = r2_score(y_train, y_pred_train)

# Métricas test
test_rmse_final = np.sqrt(mean_squared_error(y_test, y_pred_test))
test_mae_final = mean_absolute_error(y_test, y_pred_test)
test_r2_final = r2_score(y_test, y_pred_test)

# Overfitting ratio
overfitting_ratio_final = test_rmse_final / train_rmse_final

print(f"   Train RMSE: {train_rmse_final:,.0f}")
print(f"   Test RMSE: {test_rmse_final:,.0f}")
print(f"   Test MAE: {test_mae_final:,.0f}")
print(f"   Test R²: {test_r2_final:.4f}")
print(f"   Overfitting ratio: {overfitting_ratio_final:.2f}x")

# ═══════════════════════════════════════════════════════════════════════════
# GUARDAR METADATOS
# ═══════════════════════════════════════════════════════════════════════════
print("\n📋 Guardando metadatos del modelo...")

metadata = {
    "version": "2.0",
    
    "model_info": {
        "nombre": best_model_name if 'best_model_name' in locals() else "Gradient Boosting",
        "tipo": "regresion",
        "framework": "scikit-learn",
        "version_sklearn": "1.3.0",
        "descripcion": "Modelo de predicción de turistas en Mendoza con categorías normalizadas"
    },
    
    "metricas": {
        "test_rmse": float(test_rmse_final),
        "test_mae": float(test_mae_final),
        "test_r2": float(test_r2_final),
        "train_rmse": float(train_rmse_final),
        "train_r2": float(train_r2_final),
        "overfitting_ratio": float(overfitting_ratio_final)
    },
    
    "dataset_info": {
        "n_registros_train": int(len(X_train)),
        "n_registros_test": int(len(X_test)),
        "n_registros_total": int(len(df)),
        "n_features_input": int(len(numeric_features) + len(categorical_features)),
        "fecha_corte_train_test": "Split temporal 80/20",
        "periodo_datos": {
            "inicio": f"{int(df['año'].min())}-{int(df['mes'].min()):02d}",
            "fin": f"{int(df['año'].max())}-{int(df['mes'].max()):02d}"
        }
    },
    
    "features": {
        "numericas": numeric_features,
        "categoricas": categorical_features,
        "categorias_unicas": categorias_limpias
    },
    
    "preprocessing": {
        "numeric_imputer": "median",
        "numeric_scaler": "StandardScaler",
        "categorical_imputer": "constant (missing)",
        "categorical_encoder": "OneHotEncoder"
    },
    
    "training_info": {
        "fecha_entrenamiento": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "autor": "Juliccc",
        "entrega": "4 - Visualización e Integración",
        "random_state": RANDOM_STATE if 'RANDOM_STATE' in locals() else 42
    },
    
    "limpieza_datos": {
        "categorias_normalizadas": True,
        "mapeos_aplicados": {
            "pais_origen": {
                "EEUU variantes": "EEUU, Canadá y México",
                "Europa variantes": "Europa y Resto del Mundo",
                "Resto América variantes": "Resto de América"
            }
        }
    }
}

metadata_filename = 'models/modelo_metadata.json'
with open(metadata_filename, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

print(f"✅ Metadatos guardados en: {metadata_filename}")
print(f"   Tamaño: {os.path.getsize(metadata_filename) / 1024:.2f} KB")

# ═══════════════════════════════════════════════════════════════════════════
# GUARDAR ESTADÍSTICAS DEL TARGET
# ═══════════════════════════════════════════════════════════════════════════
print("\n📊 Guardando estadísticas del target...")

target_stats = {
    "train": {
        "mean": float(y_train.mean()),
        "std": float(y_train.std()),
        "min": float(y_train.min()),
        "max": float(y_train.max()),
        "median": float(y_train.median()),
        "q25": float(y_train.quantile(0.25)),
        "q75": float(y_train.quantile(0.75))
    },
    "test": {
        "mean": float(y_test.mean()),
        "std": float(y_test.std()),
        "min": float(y_test.min()),
        "max": float(y_test.max()),
        "median": float(y_test.median()),
        "q25": float(y_test.quantile(0.25)),
        "q75": float(y_test.quantile(0.75))
    }
}

stats_filename = 'models/target_stats.json'
with open(stats_filename, 'w', encoding='utf-8') as f:
    json.dump(target_stats, f, indent=2)

print(f"✅ Estadísticas guardadas en: {stats_filename}")

# ═══════════════════════════════════════════════════════════════════════════
# VERIFICACIÓN
# ═══════════════════════════════════════════════════════════════════════════
print("\n🧪 Verificando archivos...")

try:
    # Cargar modelo
    modelo_test = joblib.load(model_filename)
    pred_test = modelo_test.predict(X_test.iloc[:1])
    print(f"✅ Modelo carga correctamente (predicción test: {pred_test[0]:,.0f})")
    
    # Verificar metadata
    with open(metadata_filename, 'r', encoding='utf-8') as f:
        meta_test = json.load(f)
    
    if 'categorias_unicas' in meta_test['features']:
        print(f"✅ Metadata con categorías limpias verificado")
        for col, cats in meta_test['features']['categorias_unicas'].items():
            print(f"   • {col}: {len(cats)} categorías")
    else:
        print(f"⚠️ ADVERTENCIA: Falta 'categorias_unicas' en metadata")
        
except Exception as e:
    print(f"❌ Error en verificación: {str(e)}")

# ═══════════════════════════════════════════════════════════════════════════
# GUARDAR DATOS DE EJEMPLO
# ═══════════════════════════════════════════════════════════════════════════
print("\n📝 Guardando datos de ejemplo...")

sample_data = df.sample(min(100, len(df)), random_state=42)[
    numeric_features + categorical_features + ['turistas']
]

sample_filename = 'models/sample_data.csv'
sample_data.to_csv(sample_filename, index=False, encoding='utf-8')

print(f"✅ Datos de ejemplo guardados: {sample_filename}")
print(f"   {len(sample_data)} registros")

# ═══════════════════════════════════════════════════════════════════════════
# RESUMEN FINAL
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "="*80)
print("✅ EXPORTACIÓN COMPLETADA (VERSIÓN 2.0)")
print("="*80)

print(f"\n📦 Archivos generados:")
archivos = [
    (model_filename, "Modelo"),
    (metadata_filename, "Metadata con categorías limpias"),
    (stats_filename, "Estadísticas"),
    (sample_filename, "Datos ejemplo")
]

for archivo, desc in archivos:
    size_kb = os.path.getsize(archivo) / 1024
    print(f"   ✓ {desc}: {size_kb:.2f} KB")

print(f"\n📊 Categorías incluidas:")
for col, cats in categorias_limpias.items():
    print(f"\n   {col}: {len(cats)} categorías")
    for cat in cats:
        print(f"      • {cat}")


print("\n" + "="*80)


💾 EXPORTACIÓN DEL MODELO PARA PRODUCCIÓN (v2.0)

📁 Creando estructura de directorios...
✅ Directorio 'models/' ya existe

💾 Guardando pipeline del modelo...
✅ Modelo guardado en: models/modelo_turismo_mendoza_final.pkl
   Tamaño: 316.25 KB

🔍 Extrayendo categorías únicas del dataset limpio...

   📊 pais_origen: 8 categorías
      1. Bolivia                                       (  124 registros,  4.98%)
      2. Brasil                                        (  322 registros, 12.94%)
      3. Chile                                         (  394 registros, 15.84%)
      4. EEUU, Canadá y México                         (  322 registros, 12.94%)
      5. Europa y Resto del Mundo                      (  638 registros, 25.64%)
      6. Paraguay                                      (  124 registros,  4.98%)
      7. Resto de América                              (  322 registros, 12.94%)
      8. Uruguay                                       (  242 registros,  9.73%)

   📊 punto_entrada: 5 ca